Search Data Processing
=========================



In [2]:
import pandas as pd
import re
import string

In [3]:
search_data = pd.read_csv("./data/sample_openData_searchTerms_clean.csv")

In [4]:
search_data.shape

(5000, 8)

In [5]:
search_data.tail()

,Search Term,Exit Page,Total Unique Searches,Results Pageviews / Search,% Search Exits,% Search Refinements,Time after Search,Average Search Depth
4995,address range,/Geographic-Locations-and-Boundaries/Streets-o...,2,1.5,0.00%,0.00%,0:02:09,8.0
4996,address shapefile,/browse/embed?category=&limit=20&limitTo=&q=ad...,2,1.5,0.00%,66.67%,0:00:18,3.0
4997,address to geographic location,/browse/embed?category=&limit=20&limitTo=&q=ad...,2,1.0,0.00%,0.00%,0:00:03,1.5
4998,address with resident names,/browse/embed?category=&limit=20&limitTo=&q=ad...,2,1.0,0.00%,0.00%,0:00:05,1.5
4999,addresse,/browse/embed?category=&limit=20&limitTo=&q=po...,2,1.0,0.00%,100.00%,0:00:03,1.5


In [6]:
#search_data[search_data['Total Unique Searches']<5]

In [8]:
search_terms = list(set(search_data['Search Term']))

In [9]:
len(search_terms)

2451

In [10]:
#print search_terms

In [11]:
search_data.ix[search_data["Search Term"] == "194415"]

,Search Term,Exit Page,Total Unique Searches,Results Pageviews / Search,% Search Exits,% Search Refinements,Time after Search,Average Search Depth
3519,194415,/Economy-and-Community/Registered-Business-Loc...,2,1,0.00%,0.00%,0:00:06,2.5


In [12]:
search_terms_data = search_data[["Search Term"]]
search_terms_data = search_terms_data.rename(columns={"Search Term": "search_term"})

In [13]:
search_terms_data.head()

,search_term
0,business
1,crime
2,crime
3,311
4,streets


In [14]:
search_terms_data['processed_data'] = search_terms_data.search_term\
                                        .apply(lambda text: text.decode('ascii' ,"ignore" ).decode('utf-8','ignore'))\
                                        .apply(lambda text: text.lower())

In [15]:
search_terms_list =  list(set(search_terms_data.processed_data))
#print search_terms_list

#### We need to generally tag searches in categories of quality

Many search terms indicate that users are not informed on what the purpose of the website is or how to properly use search

Search Tags
* Good Quality Search
    * complete words or phrases
    * minor typos
* Bad Quality
    * Addresses
    * Dates
    * Zipcodes or just a string of numbers
    * General nonsenses eg. ('></script><script>alert(1)</script>', '///', '16exc-3031') 

In [16]:
dates_r = re.compile('[0-9]/[0-9]/[0-9]')
numbers_r = re.compile('^[0-9][0-9]*[0-9]$')
html_r = re.compile('^<.*>$')


In [17]:
filter(dates_r.match, search_terms_list)

[u'9/1/01', u'3/1/01']

In [18]:
print filter(html_r.match, search_terms_list)

[u'</script><script>alert(1)</script>']


In [19]:
# removing punctuation

def removePunctuation(text):

    for c in string.punctuation:
        text = text.replace(c,"").strip().lower()
    return text

In [20]:
# iterative process

def text_processing(search):
    
    return [removePunctuation(i) for i in search]
        
        

In [21]:
search = text_processing(search_terms_list)
#search

In [22]:
regex = "\d{1,4}.?\d{0,4}\s[a-zA-Z|\d+]{2,30}\s[a-zA-Z]{2,15}"

f = [re.findall(regex, i) for i in search
     if re.findall(regex, i)!= [] 
     if re.findall(regex, i)[0][:3] != '311'
     ]

# http://regexlib.com/REDetails.aspx?regexp_id=430

In [23]:
len(f)

409

In [24]:
year = [str(j) for j in range(2000,2017)]

addresses = [i for i in f if i[0][:4] not in year ]
#addresses